In [1]:
#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
//using BoSSS.XNSE_Solver;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using MomentFittingVariants = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "VanishingSphere2D" : buildname; //Popcorn2D  VanishingSphere2D
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "convergenceAggSteadyStokesExactBC" : dbname; //Collison´2
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
using BoSSS.Solution.Gnuplot;


In [4]:
//ExecutionQueues
var myBatch = ExecutionQueues[5];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

Project name is set to 'VanishingSphere2D_convergenceAggSteadyStokesExactBC'.
Opening existing database '\\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC'.


{ Session Count = 90; Grid Count = 10; Path = \\dc3\userspace\toprak\BoSSS-ConvSteadyFinal\VanishingSphere2D_convergenceAggSteadyStokesExactBC }

In [5]:
myDB.Sessions.Where(s => s.SuccessfulTermination == true)

#0: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.2_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:03:04 PM	4e6c9e76...
#1: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.5_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:04:14 PM	136662d2...
#2: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.4_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:03:59 PM	2e86a22b...
#3: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k3_agg0.3_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:03:43 PM	99b0760f...
#4: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg0.5_AMRFalse_LBFalse_atZ_r0.6_W6.94_2D_StokesdRdt0.00_MUMPS_Steady_ct1	8/20/2024 1:02:52 PM	fa95be63...
#5: VanishingSphere2D_convergenceAggSteadyStokesExactBC	SolverTrue_J57600_k2_agg

In [6]:
int timeStep = -1;
// BoSSSshell.WorkflowMgm.hConvergence.Update(timeStep);

In [7]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
Tab.Columns.Summary()

Count: 231
IsReadOnly: False
IsSynchronized: False
SyncRoot: System.Data.DataColumnCollection


In [8]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns( "AgglomerationThreshold", "DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2err_"+ "VelocityX", "L2Err_" +"Pressure"); // 

// Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> 
//             Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0
//                 // Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh
//                 // && Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsCoarsestMesh
//                 // && Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != 900
//                 );
Tab2.Print();  
Tab2.SaveToFile(table_name + $"_at{timeStep}_summary.json");

    AgglomerationThreshold DGdegree:Velocity* Grid:hMin            Grid:NoOfCells L2err_VelocityX        L2err_Pressure         
0:  0.2                    3                  0.008333333333333304 57600          3.697938199644299E-09  2.5588808254847334E-07 
1:  0.5                    3                  0.008333333333333304 57600          5.216859594367646E-09  2.5425053894028194E-07 
2:  0.4                    3                  0.008333333333333304 57600          4.5482850658530185E-09 2.506460091890163E-07  
3:  0.3                    3                  0.008333333333333304 57600          4.0801030801596335E-09 2.4923627615002925E-07 
4:  0.5                    2                  0.008333333333333304 57600          1.2007818915739127E-06 1.3790259368558115E-05 
5:  0.3                    2                  0.008333333333333304 57600          1.1182617957953E-06    1.1875199557467882E-05 
6:  0.4                    2                  0.008333333333333304 57600          1.1514820437518

In [9]:
var grids = myDB.Sessions.Select(s => s.Timesteps.First().Grid.NumberOfCells);

int NCellsFinestMesh =  57600; //grids.Max(); // myDB.Sessions.Select(s => (GridData)s.Timesteps.First().Grid) // 57600; //int 14400;
int NCellsCoarsestMesh =  225; //grids.Min(); 
Math.Sqrt(NCellsFinestMesh)

240

In [10]:
// Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> 
//                 Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh
//                  && Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0.1
                
//                  );

In [11]:
//grids

In [12]:
using BoSSS.Solution.Gnuplot;
using System.Linq;

In [21]:
int counter = 0;
string[] varNames = new string[]{"VelocityX", "VelocityY", "Pressure", "Pressure"};

IEnumerable<KeyValuePair<string, double>>[] RegressionS = new IEnumerable<KeyValuePair<string, double>>[varNames.Length];

    Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 2];
    int cnt2 = 0;
    for (int iCol = 0; iCol < 2; iCol++) {
        for (int iRow = 0; iRow < 2; iRow++) {
            var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
            var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
            PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle };
            
            // BoSSS data
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.PointType =  allPointTypes[cnt2 % allPointTypes.Length];
            fmt.DashType = DashTypes.Solid;            
            
            string dgDegStr = (varNames[cnt2] == "VelocityX"  || varNames[cnt2] == "VelocityY" || varNames[cnt2] == "Residual-ContiEq") ? "DGdegree:Velocity*":  "DGdegree:"+varNames[cnt2] ;
            var Tab = BoSSSshell.WorkflowMgm.SessionTable;
            var Tab2 = Tab.ExtractColumns( "AgglomerationThreshold", dgDegStr,"Grid:hMin","Grid:NoOfCells" ,"L2err_"+  varNames[cnt2]);
            Tab2.Summary();
            //Extract last point, the one against we are comparing our solutions with
            Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> 
            Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0
                //&& Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh
                // && Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsCoarsestMesh
                // && Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != 900

                );
            var plot = Tab2.ToPlot("Grid:hMin","L2err_"+  varNames[cnt2],  dgDegStr);

            int cc = 0;
            foreach(var dataGroup in plot.dataGroups){
                dataGroup.Format.LineColor = allColors[cc % allColors.Length];
                dataGroup.Format.PointType = myPointTypes[cc];
                cc++;
            }            
            plot.LogX = true;
            plot.LogY = true;
            //plot.LegendFont = 10;
            //plot.LegendHorizontal = true;
            plot.Title = varNames[cnt2] ; //+ " " +table_name ;
            plot.TitleFont = 10;
            plot.Xlabel = "h";  
            plot.Ylabel = "L_2 Error";  
            PlotTable[iCol, iRow] = plot;

            RegressionS[cnt2] = plot.Regression();
            cnt2++;
        }
    }

In [22]:
PlotTable.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityX 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityY 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Pressure 
 
 
 DGdegree:Pressure0 
 
 
 DGdegree:Pressure0 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure2 
 
 
 DGdegree:Pressure2 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure1 
 
 
 DGdegree:Pressure1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Pressure 
 
 
 DGdegree:Pressure0 
 
 
 DGdegree:Pressure0 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure2 
 
 
 DGdegree:Pressure2 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:Pressure1 
 
 
 DGdegree:Pressure1

In [23]:
int cnt = 0;
foreach(var reg in RegressionS){

    Console.WriteLine(varNames[cnt]);
    
    var regB = reg.ToArray();
    for(int j = 0; j < regB.Length; j++){       
        int dgDeg = ( regB[j].Key).Last() - '0';
        Console.Write("DG degree: "+ dgDeg+ " \t");
        double slope = regB[j].Value;
        Console.Write("Slope: " +slope + "\t");

        if((slope - (dgDeg + 1) ) >= -0.2)
        Console.WriteLine("(+)"); //Console.WriteLine("Expected slope archieved");
        else
        Console.WriteLine("(-)");
    }
    cnt++;
}

VelocityX
DG degree: 1 	Slope: 1.3169697486750147	(-)
DG degree: 3 	Slope: 1.4268808534420538	(-)
DG degree: 2 	Slope: 3.5023534797322076	(+)
VelocityY
DG degree: 1 	Slope: 1.316951229526711	(-)
DG degree: 3 	Slope: 1.4953205839309185	(-)
DG degree: 2 	Slope: 3.502327211858611	(+)
Pressure
DG degree: 0 	Slope: 1.4366021683183565	(+)
DG degree: 2 	Slope: -0.05629728467916807	(-)
DG degree: 1 	Slope: 2.244405155119123	(+)
Pressure
DG degree: 0 	Slope: 1.4366021683183565	(+)
DG degree: 2 	Slope: -0.05629728467916807	(-)
DG degree: 1 	Slope: 2.244405155119123	(+)


In [16]:
RegressionS

index,value
0,"[ [DGdegree:Velocity*1, 1.3169697486750147], [DGdegree:Velocity*3, 1.4268808534420538], [DGdegree:Velocity*2, 3.5023534797322076] ]"
1,"[ [DGdegree:Velocity*1, 1.316951229526711], [DGdegree:Velocity*3, 1.4953205839309185], [DGdegree:Velocity*2, 3.502327211858611] ]"
2,"[ [DGdegree:Pressure0, 1.4366021683183565], [DGdegree:Pressure2, -0.05629728467916807], [DGdegree:Pressure1, 2.244405155119123] ]"
3,"[ [DGdegree:Pressure0, 1.4366021683183565], [DGdegree:Pressure2, -0.05629728467916807], [DGdegree:Pressure1, 2.244405155119123] ]"
